In [2]:
%pip install pandas
%pip install selenium
%pip install Pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [171]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

def extract_emails(website_url):
    browser.get(website_url)
    
    # Get the regular expressions pattern to match on
    email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}"
    # Get the html content for the page
    html = browser.page_source
    # Get a list of emails
    emails = re.findall(email_pattern, html)
    # Get a unique array of the emails on the page
    unique_emails_arr = pd.DataFrame(columns=['email'],data=emails)['email'].unique()
    # Convert to string with a comma delimiting them
    unique_emails_str = ','.join(unique_emails_arr)
    return unique_emails_str


# Define a function to extract search results for a given search URL, location, and industry
def extract_search_results(search_url,location,industry):

    # Navigate to the search URL
    browser.get(search_url)
    wait = WebDriverWait(browser, 10)
    
    # Wait for the page to load
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "hfpxzc")))
        
    # Find the element to start scrolling from
    start_scroll_element = browser.find_element(By.CLASS_NAME,"hfpxzc")
    scroll_origin = ScrollOrigin.from_element(start_scroll_element)
    
    # Perform scrolling action to reveal more search results
    for i in range(max_num_scrolls):
        ActionChains(browser).scroll_from_origin(scroll_origin,0,1500*(i+1)).perform()
        
        # Possibly find a way to use selenium WebDriverWait 
        time.sleep(3)
        
    # Inner function to extract information about a specific place from its page
    def extract_place_info(place_url):
        browser.get(place_url)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "lMbq3e")))
                
        # Extract the header information about the place
        header_text = browser.find_element(By.CLASS_NAME,'lMbq3e').text
        header_list = header_text.split("\n")
        place_name = header_list[0]
        reviews_stars = header_list[1]
        num_of_reviews = header_list[2].strip('(').strip(')')
        place_description = header_list[3]
        
        # Extract additional information like address, phone, and website
        info_elements = browser.find_elements(By.CLASS_NAME, "CsEnBe")
        website = None
        phone = None
        address = None
        emails = None
        
        # TODO: Find a way to make this robust against locale differences
        if english:  # Handle being in Mexico
            for info_element in info_elements:
                aria_label = info_element.get_attribute("aria-label")
                if 'Address:' in aria_label:
                    address = info_element.text
                if 'Phone:' in aria_label:
                    phone = info_element.text
                if 'Website:' in aria_label:
                    website = info_element.get_attribute("href")
        else:  # If Spanish
            for info_element in info_elements:
                aria_label = info_element.get_attribute("aria-label")
                if 'Dirección:' in aria_label:
                    address = info_element.text.split('\n')[1]
                if 'Teléfono:' in aria_label:
                    phone = info_element.text
                if 'Sitio web:' in aria_label:
                    website = info_element.get_attribute("href")
        
        # Check if the place is a duplicate
        if address in all_addresses:
            if place_name in all_place_names:
                return "duplicate"
        else:
            all_addresses.append(address)
            all_place_names.append(place_name)
        
        if website is not None:
            emails = extract_emails(website)
        
        # Compile the extracted information into a list
        place_info = [
            industry,
            location,
            place_name,
            place_description,
            reviews_stars,
            num_of_reviews,
            address,
            phone,
            emails,
            website
        ]
        
        print(place_info)
        
        return place_info
    
    # Find all elements that represent places in the search results
    place_elements = browser.find_elements(By.CLASS_NAME, "hfpxzc")
        
    # Extract the URLs of all places
    place_urls = [element.get_attribute("href") for element in place_elements]
    
    place_info_list = []
    
    # Loop through the URLs to extract information for each place
    for url in place_urls:
        try:
        # if True:  # TODO: There seems to be an issue in extract_place_info 
                  # that causes an "argument of type 'NoneType' is not iterable" error
            temp_place_info = extract_place_info(url)
            
            if temp_place_info == "duplicate":  # Skip if the place is a duplicate
                continue
            
            place_info_list.append(temp_place_info)
            
        except Exception as e:
            print(f"Error extracting place with url: {url}. Error: {e}")
        
        # Stop if max_places_to_find places have been found
        if len(place_info_list) >= max_places_to_find:
            print('Breaking out of the for loop because {max_places_to_find} places were found.')
            break

    return place_info_list


# Prepare to compile information for multiple locations and industries
final_places_list = []
all_addresses = []
all_place_names = []

# Spanish to handle Mexico
english = False

# Parameters for the search
max_places_to_find = 1000  # Maximum number of places to find for each search
max_num_scrolls = 10  # Maximum number of times to scroll to reveal more search results

# Define the locations and industries to search for
locations = ['Los Angeles, CA',] # 'San Francisco, CA','New York, NY','Jacksonville, FL','Portland, OR','Denver, CO']
industries = ['Churros', 'Churro', 'Churrerria']  # Add more industries to search for
# 'Physical Therapy',
# 'HVAC',
# 'Veterinary Services',
# 'Plumbing',
# 'Insurance Brokerages',
# 'Landscaping',
# 'Automotive Repair',
# 'Chiropractic']

# Initialize the WebDriver Options 
options = Options()
options.add_argument("--headless")
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})  # Set language preferences

# Loop through each industry and location, perform searches, and compile results
for industry in industries:
    print(industry)
    for location in locations:  
        # Initialize the WebDriver for Chrome
        browser = webdriver.Chrome(options=options)
        
        try:
            print(location)
            search_query = f"{industry} in {location}"
            print(search_query)
            search_url = f"https://www.google.com/maps/search/{search_query}"
            combo_place_info_list = extract_search_results(search_url,location,industry)
            pd.DataFrame(combo_place_info_list).to_csv(f'{industry} Export.csv',index = False)
            final_places_list.extend(combo_place_info_list)
        except Exception as e:
            print(e)
        
        finally:
            browser.close()

browser.quit()


Churros
Los Angeles, CA
Churros in Los Angeles, CA
['Churros', 'Los Angeles, CA', 'Mr. Churro', '·$', '4.3', '493', 'E-12 Olvera St # C, Los Angeles, CA 90012, Estados Unidos', '', None, None]
['Churros', 'Los Angeles, CA', 'Ridges Churro Bar', 'Tienda de postres', '2.8', '4', 'Los Angeles, CA 90014, Estados Unidos', None, '', 'http://instagram.com/ridgeschurrobar']
['Churros', 'Los Angeles, CA', 'Churro Factory', 'Churrería', '5.0', '1', '5333 Zoo Dr, Los Angeles, CA 90027, Estados Unidos', None, None, None]
['Churros', 'Los Angeles, CA', 'La Churreria', 'Tienda de postres', '3.7', '3', '532 S Olive St, Los Angeles, CA 90013, Estados Unidos', None, None, None]
Error extracting place with url: https://www.google.com/maps/place/Churros+Calientes/data=!4m7!3m6!1s0x80c2bb6f92c3175d:0x735a857a04b3ab45!8m2!3d34.0453847!4d-118.4528003!16s%2Fg%2F1tdmvwy4!19sChIJXRfDkm-7woARRauzBHqFWnM?authuser=0&hl=es-419&rclk=1. Error: argument of type 'NoneType' is not iterable
['Churros', 'Los Angeles, CA'

# Data Cleaning and Testing

In [172]:
# Create a DataFrame from the compiled list and save it to a CSV file
places_df = pd.DataFrame(columns=['Industry', 'Location', 'Name', 'Description', 'Stars (out of 5)', 'Number of Reviews', 'Address', 'Phone', 'Emails','Website'], data=final_places_list)
places_df.to_csv('More Churro Places.csv', index=False)

In [9]:
len(final_places_list)
# print(final_places_list)


3

In [37]:
fixed_addresses = []

for address in places_df['Address']:
    fixed_addresses.append(address.split('\n')[1])

In [39]:
fixed_addresses

['231 Border St B, Boston, MA 02128, Estados Unidos',
 '9 Channel Center St, Boston, MA 02210, Estados Unidos',
 '400 Commonwealth Ave 3rd floor, Boston, MA 02215, Estados Unidos',
 '1 Huntington Ave B, Boston, MA 02116, Estados Unidos',
 '1 McKinley Square Lobby B, Boston, MA 02109, Estados Unidos',
 '575 Boylston St 7th floor, Boston, MA 02116, Estados Unidos',
 '177 Tremont St #6, Boston, MA 02111, Estados Unidos',
 '77 Beacon St, Boston, MA 02108, Estados Unidos',
 '31 State St 7th Floor, Boston, MA 02109, Estados Unidos',
 '1610 Tremont St, Boston, MA 02120, Estados Unidos',
 '37 Newbury St #3, Boston, MA 02116, Estados Unidos',
 '36 Chauncy St, Boston, MA 02111, Estados Unidos',
 '591 E Broadway, Boston, MA 02127, Estados Unidos',
 '1313 Washington St, Boston, MA 02118, Estados Unidos',
 '272 W Broadway, Boston, MA 02127, Estados Unidos',
 '800 Boylston St 2nd floor, Boston, MA 02199, Estados Unidos',
 '22 Traveler St, Boston, MA 02118, Estados Unidos',
 '46 Farnsworth St, Boston

In [44]:
places_df.to_csv('Places Information Fixed.csv', index=False)

In [22]:
addresses = places_df['Address'].dropna().to_numpy()

In [24]:
import numpy as np

unique_addies = np.unique(addresses)

In [25]:
unique_addies.shape

(24,)

In [30]:
filtered_df = places_df[places_df['Address'].isin(unique_addies)]


In [33]:
filtered_df['Address'].to_numpy().shape

(45,)

In [44]:
places_dropna = places_df.loc[places_df['Address'].dropna(inplace=False).index]

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'

In [51]:
places_df.shape

(47, 10)

In [54]:
# places_df

In [59]:
df_unique = places_df.drop_duplicates(subset='Address', keep='first')

In [62]:
df_unique.dropna(subset='Address', inplace=True)

C:\Users\wilso\AppData\Local\Temp\ipykernel_41672\1627190734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.dropna(subset='Address', inplace=True)


In [65]:
# df_unique.to_csv('Unique Churro Places LA.csv', index=False)

In [ ]:
Churros
Los Angeles, CA
Churros in Los Angeles, CA
0
1
2
3
4
30
['Churros', 'Los Angeles, CA', 'Mr. Churro', '·$', '4.3', '493', 'E-12 Olvera St # C, Los Angeles, CA 90012, Estados Unidos', '\ue0b0\n+1 213-680-9036', None, None]
['Churros', 'Los Angeles, CA', 'Churros Estilo Guadalajara', 'Tienda de postres', '4.8', '10', '4025 S Central Ave, Los Angeles, CA 90011, Estados Unidos', '\ue0b0\n+1 323-604-8135', None, None]
['Churros', 'Los Angeles, CA', 'Ridges Churro Bar', 'Tienda de postres', '2.8', '4', 'Los Angeles, CA 90014, Estados Unidos', None, '', 'http://instagram.com/ridgeschurrobar']
['Churros', 'Los Angeles, CA', 'Salinas Churros', 'Churrería', '3.7', '34', '918 S Lorena St, Los Angeles, CA 90023, Estados Unidos', None, '', 'https://instagram.com/salinaschurros?igshid=YmMyMTA2M2Y=']
['Churros', 'Los Angeles, CA', 'Churros y platanos', 'Churrería', '5.0', '17', 'Los Angeles, CA 90034, Estados Unidos', None, None, None]
['Churros', 'Los Angeles, CA', 'La Churreria', 'Tienda de postres', '3.7', '3', '532 S Olive St, Los Angeles, CA 90013, Estados Unidos', None, None, None]
Error extracting place with url: https://www.google.com/maps/place/Churros+Calientes/data=!4m7!3m6!1s0x80c2bb6f92c3175d:0x735a857a04b3ab45!8m2!3d34.0453847!4d-118.4528003!16s%2Fg%2F1tdmvwy4!19sChIJXRfDkm-7woARRauzBHqFWnM?authuser=0&hl=es-419&rclk=1. Error: argument of type 'NoneType' is not iterable
['Churros', 'Los Angeles, CA', 'Churros El Gorilla', 'Restaurante de postres', '5.0', '11', '5525 Sunset Blvd, Los Angeles, CA 90028, Estados Unidos', '\ue0b0\n+1 747-258-8650', None, None]
Error extracting place with url: https://www.google.com/maps/place/Juanitos+churros/data=!4m7!3m6!1s0x80c2c71905108697:0x7db9b984196aed10!8m2!3d34.0395879!4d-118.2133021!16s%2Fg%2F11txfljlr3!19sChIJl4YQBRnHwoAREO1qGYS5uX0?authuser=0&hl=es-419&rclk=1. Error: list index out of range
['Churros', 'Los Angeles, CA', 'Churrito loco', '·$$', '4.6', '38', '5161 Pomona Blvd # 105, East Los Angeles, CA 90022, Estados Unidos', None, 'eatchurritoloco@gmail.com,605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com,8eb368c655b84e029ed79ad7a5c1718e@sentry.wixpress.com', 'http://www.eatchurritoloco.com/']
['Churros', 'Los Angeles, CA', 'Churros El Bochito', 'Restaurante', '4.2', '20', '1157 Cypress Ave, Los Angeles, CA 90065, Estados Unidos', '\ue0b0\n+1 213-820-0172', None, None]
['Churros', 'Los Angeles, CA', 'crepes n churros', 'Crepería', '3.5', '11', '2744 1st St, Los Angeles, CA 90033, Estados Unidos', '\ue0b0\n+1 323-338-1863', None, None]
Breaking out of the for loop because {max_places_to_find} places were found.
Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF727FB1502+60802]
	(No symbol) [0x00007FF727F2AC02]
	(No symbol) [0x00007FF727DE7B13]
	(No symbol) [0x00007FF727E28B6B]
	(No symbol) [0x00007FF727E29C21]
	GetHandleVerifier [0x00007FF7282B411D+3217821]
	GetHandleVerifier [0x00007FF7282F60B7+3488055]
	GetHandleVerifier [0x00007FF7282EF03F+3459263]
	GetHandleVerifier [0x00007FF72806B846+823494]
	(No symbol) [0x00007FF727F35F9F]
	(No symbol) [0x00007FF727F30EC4]
	(No symbol) [0x00007FF727F31052]
	(No symbol) [0x00007FF727F218A4]
	BaseThreadInitThunk [0x00007FF86FC9257D+29]
	RtlUserThreadStart [0x00007FF870D2AA48+40]



In [8]:
# Get Google maps platform api key

from dotenv import load_dotenv
import os


# Load environment variables from .env file
load_dotenv()

# Access your API key using the os.environ dictionary
api_key = os.getenv("GOOGLE_MAPS_PLATFORM_API_KEY")


In [92]:
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import time
import numpy as np
import pandas as pd

# Load environment variables (including the API key)
load_dotenv()

# Get the API key from environment variables
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")


def get_next_2am_unix_timestamp():
    """Get the Unix timestamp for the next 2 a.m."""
    now = datetime.now()
    next_2am = now.replace(hour=2, minute=0, second=0, microsecond=0)

    # If it's already past 2 a.m. today, move to the next day
    if now >= next_2am:
        next_2am += timedelta(days=1)

    return int(time.mktime(next_2am.timetuple()))




def is_within_time(origins: np.ndarray, destinations: np.ndarray) -> np.ndarray:
    """Check if the travel time from the origin to each destination is within a specified time.
    
    Parameters:
        origins (ndarray(n,)): The starting locations.
        destinations (ndarray(m,)): A numpy array of destination addresses.
        time (int): The maximum travel time in seconds.
        
    Returns:
        ndarray(n, m): A numpy array of boolean values indicating if the travel time to each destination is within the specified time.
    """
    n, m = len(origins), len(destinations)
    times_arr = np.zeros((n, m), dtype=float)
    
    if type(origins) != list and type(origins) != np.ndarray:
        raise TypeError("Origins must be a numpy array.")
    else:
        destinations = np.array(destinations)
            
    if type(destinations) != list and type(destinations) != np.ndarray:
        raise TypeError("Destinations must be a numpy array.")
    else:
        destinations = np.array(destinations)
        
    if type(origins) == list:
        origins = np.array(origins)
    if type(destinations) == list:
        destinations = np.array(destinations)
    
    for i, origin in enumerate(origins):
        
        base_url = "https://maps.googleapis.com/maps/api/directions/json"
        within_time = []

        # Calculate the Unix timestamp for the next 2 a.m.
        departure_time = str(get_next_2am_unix_timestamp())

        for j, destination in enumerate(destinations):
            params = {
                "origin": origin,
                "destination": destination,
                "departure_time": departure_time,  # Dynamically calculated 2 a.m. timestamp
                "key": GOOGLE_MAPS_API_KEY
            }

            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                data = response.json()

                # Check if the API returned a valid route
                if data["status"] == "OK":
                    duration_seconds = data["routes"][0]["legs"][0]["duration"]["value"]
                    times_arr[i, j] = duration_seconds  # Store the travel time

                else:
                    print(f"No valid route for {destination}. \nStatus: {data['status']}")
            else:
                raise Exception(f"Google Maps API error: {response.status_code}")
    
    df_times = pd.DataFrame(data=times_arr, columns=destinations, index=origins)
    
    return df_times


In [58]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy as np
import pandas as pd


# Function to convert addresses to latitude/longitude
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        raise ValueError(f"Could not geocode address: {address}")


def is_within_geodesic_distance(origins: np.ndarray, destinations: np.ndarray, max_distance: float) -> pd.DataFrame:
    """Check if the geodesic travel time from each origin to each destination is within a specified time.

    Parameters:
        origins (ndarray): Array of starting addresses.
        destinations (ndarray): Array of destination addresses.
        max_distance (float): Maximum distance in miles.

    Returns:
        DataFrame: A DataFrame indicating which destinations are within the specified travel time from each origin.
    """
    # Convert travel time to maximum distance in km
    max_distance_km = max_distance * 1.60934

    # Geocode the origins and destinations
    origins_coords = [geocode_address(origin) for origin in origins]
    destinations_coords = [geocode_address(destination) for destination in destinations]

    n, m = len(origins), len(destinations)
    within_distance_final = np.zeros((n, m), dtype=bool)

    for i, origin_coord in enumerate(origins_coords):
        within_distance = []
        for destination_coord in destinations_coords:
            distance_km = geodesic(origin_coord, destination_coord).kilometers
            within_distance.append(distance_km <= max_distance_km)

        within_distance_final[i, :] = np.array(within_distance)

    return pd.DataFrame(data=within_distance_final, columns=destinations, index=origins)


# Example Usage
origin_addresses = np.array([
    "1600 Amphitheatre Parkway, Mountain View, CA",
    "1 Infinite Loop, Cupertino, CA"
])
destination_addresses = np.array([
    "San Francisco, CA",
    "San Jose, CA",
    "Palo Alto, CA"
])

# Check destinations within a 10-minute drive (assuming an average speed of 60 km/h)
max_distance = 4
result = is_within_geodesic_distance(origin_addresses, destination_addresses, max_distance)
print(result)


GeocoderInsufficientPrivileges: Non-successful status code 403

In [93]:
df = pd.read_csv('Unique Churro Places LA.csv')

origin_list = [
    "2300 Harbor Blvd, Costa Mesa, CA 92626, United States",
    "1524 Sunset Blvd, Los Angeles, CA 90026, United States",
    "12405 Washington Blvd, Los Angeles, CA 90066, United States",
    "131 N Larchmont Blvd, Los Angeles, CA 90004, United States",
    "1534 Montana Ave, Santa Monica, CA 90403, USA",
    "4455 Los Feliz Blvd, Los Angeles, CA 90027, United States",
    "100 S Avenue 64, Highland Park, CA 90042, USA",
    "600 N Brand Blvd, Glendale, CA 91203, USA",
    "9615 S Santa Monica Blvd, Beverly Hills, CA 90210, USA",
    "96 E Colorado Blvd, Pasadena, CA 91105, United States",
]

churro_list = list(df['Address'])

times_df = is_within_time(origin_list, churro_list)


In [94]:
# times_df.to_csv('times_df.csv', index=False)


In [127]:
max_time = 900

is_within_time = times_df.values < max_time

is_within_time.sum()

20

In [129]:
i_arr, j_arr = np.nonzero(is_within_time)

In [131]:
print(len(j_arr))

20


In [130]:
print(len(np.unique(j_arr)))

10


In [132]:
print(i_arr)
print(j_arr)

[1 1 1 1 1 1 1 2 2 3 5 5 5 6 6 6 6 7 7 8]
[ 0  1  2  5  6  8 11 12 15  3  3  5 11  0  2  5  8  5 11 12]


In [166]:
origin_names = [
    "Culver city",
    "Hancock Park", 
    "Griffith Park",
    "Highland Park", 
    "Palisades Park",
    "Glendale",
    "Beverly Hills",
    "Pasadena",
]

In [165]:
new_i_list = []
new_j_list = []

for k in range(len(churro_list)):
    if k not in j_arr:
        continue
    
    same_loc = i_arr[j_arr == k]
    
    closest_dist = np.inf
    
    for loc in same_loc:
        if times_df.iloc[loc, k] < closest_dist:
            closest_dist = times_df.iloc[loc, k]
            closest_loc = loc
    
    new_i_list.append(closest_loc)
    new_j_list.append(k)

sort_mask = np.argsort(new_i_list)
new_i_arr = np.array(new_i_list)[sort_mask]
new_j_arr = np.array(new_j_list)[sort_mask]

In [167]:
comb_arr = list(map(tuple, np.vstack((new_i_arr, new_j_arr)).T))


In [168]:
occurence_list = []
c = 0

for i, j in comb_arr:
    
    occurence_list.append(j)
    
    print(f"Origin Name: {origin_names[i-1]}")
    print(f"Origin Address: {origin_list[i]}")
    print(f"Destination Name: {df['Name'][j]}")
    print(f"Destination Address: {churro_list[j]}")
    print("")
    
    # Count how many unique places are within the time limit
    c += 1

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Mr. Churro
Destination Address: E-12 Olvera St # C, Los Angeles, CA 90012, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Ridges Churro Bar
Destination Address: Los Angeles, CA 90014, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: La Churreria
Destination Address: 532 S Olive St, Los Angeles, CA 90013, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: crepes n churros
Destination Address: 2744 1st St, Los Angeles, CA 90033, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Salinas Churros
Destination Address: 918 S Lorena St, Los Angeles, CA 90023, Estad

In [ ]:
"""

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Mr. Churro
Destination Address: E-12 Olvera St # C, Los Angeles, CA 90012, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Ridges Churro Bar
Destination Address: Los Angeles, CA 90014, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: La Churreria
Destination Address: 532 S Olive St, Los Angeles, CA 90013, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: crepes n churros
Destination Address: 2744 1st St, Los Angeles, CA 90033, Estados Unidos

Origin Name: Culver city
Origin Address: 1524 Sunset Blvd, Los Angeles, CA 90026, United States
Destination Name: Salinas Churros
Destination Address: 918 S Lorena St, Los Angeles, CA 90023, Estados Unidos

Origin Name: Hancock Park
Origin Address: 12405 Washington Blvd, Los Angeles, CA 90066, United States
Destination Name: Azucanela Churros & Coffee - Culver City (Westfield Mall)
Destination Address: Westfield Mall(2nd Floor next to macys, 6000 Sepulveda Blvd #2507, Culver City, CA 90230, Estados Unidos

Origin Name: Palisades Park
Origin Address: 4455 Los Feliz Blvd, Los Angeles, CA 90027, United States
Destination Name: Churros El Gorilla
Destination Address: 5525 Sunset Blvd, Los Angeles, CA 90028, Estados Unidos

Origin Name: Glendale
Origin Address: 100 S Avenue 64, Highland Park, CA 90042, USA
Destination Name: Churros El Bochito
Destination Address: 1157 Cypress Ave, Los Angeles, CA 90065, Estados Unidos

Origin Name: Beverly Hills
Origin Address: 600 N Brand Blvd, Glendale, CA 91203, USA
Destination Name: Churro Factory
Destination Address: 5333 Zoo Dr, Los Angeles, CA 90027, Estados Unidos

Origin Name: Pasadena
Origin Address: 9615 S Santa Monica Blvd, Beverly Hills, CA 90210, USA
Destination Name: Churros y platanos
Destination Address: Los Angeles, CA 90034, Estados Unidos



"""